In [1]:
# Download data
%run "../dataio/gtsrb_download.py"

2025-10-31 16:42:33,750 - INFO - Downloading GTSRB_Final_Training_Images...
2025-10-31 16:42:56,967 - INFO - Unpacking GTSRB_Final_Training_Images...
2025-10-31 16:43:04,291 - INFO - Downloading GTSRB_Final_Test_Images...
2025-10-31 16:43:12,072 - INFO - Unpacking GTSRB_Final_Test_Images...
2025-10-31 16:43:14,451 - INFO - Downloading GTSRB_Final_Test_GT...
2025-10-31 16:43:14,618 - INFO - Unpacking GTSRB_Final_Test_GT...
2025-10-31 16:43:14,623 - INFO - GTSRB data is ready.
2025-10-31 16:43:17,894 - INFO - GTSRB data cleaned and organized.


In [ ]:
%cd ..

/home/richard/Documents/academics/1a/traffic-sign-classification


In [19]:
from dataio.transforms import ToCompose, ToResize, ToRotate, ToNoise, ToTensor, ToNormalize
from dataio.gtsrb_dataset import GTSRBDataset
from dataio.dataloader import DataLoader
from nn.layers.batchnorm2d import BatchNorm2D
from nn.layers.conv2d import Conv2D
from nn.layers.dropout import Dropout
from nn.layers.flatten import Flatten
from nn.layers.linear import Linear
from nn.layers.maxpool2d import MaxPool2D
from nn.layers.sequential import Sequential
from nn.optim import Adam
from nn.loss import cross_entropy
from train import train
import numpy as np

In [20]:
# Define the transforms for training
train_transforms = ToCompose([
    ToResize(size=64),
    ToRotate(angle=15),
    ToNoise(mean=0, std=0.05),
    ToTensor(),
    ToNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# Define the transforms for validation and testing
val_transforms = ToCompose([
    ToResize(size=64),
    ToTensor(),
    ToNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# %% [markdown]
# ## Load Data

# %%
# Total number of entries in the dataset
total_entries = 51840

# Define the indices for each split
def get_train_indices():
    return list(range(int(0.7 * total_entries)))

def get_val_indices():
    start = int(0.7 * total_entries)
    end = int(0.85 * total_entries)
    return list(range(start, end))

def get_test_indices():
    start = int(0.85 * total_entries)
    return list(range(start, total_entries))

# Initialize the dataset
train_dataset = GTSRBDataset(
    root="./data/gtsrb/",
    indices=get_train_indices(),
    split="train",
    transforms=train_transforms
)
val_dataset = GTSRBDataset(
    root="./data/gtsrb/",
    indices=get_val_indices(),
    split="val",
    transforms=val_transforms
)
test_dataset = GTSRBDataset(
    root="./data/gtsrb/",
    indices=get_test_indices(),
    split="test",
    transforms=val_transforms
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [21]:
# Example architecture without ReLU
layers = [
    Conv2D(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
    BatchNorm2D(num_channels=32),
    MaxPool2D(pool_size=2, stride=2),
    Conv2D(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
    BatchNorm2D(num_channels=64),
    MaxPool2D(pool_size=2, stride=2),
    Conv2D(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
    BatchNorm2D(num_channels=128),
    MaxPool2D(pool_size=2, stride=2),
    Flatten(),
    Linear(in_features=128 * 8 * 8, out_features=512),  # Adjust input_size based on your image size and pooling layers
    Dropout(p=0.5),
    Linear(in_features=512, out_features=43)  # GTSRB has 43 classes
]

model = Sequential(layers)

In [22]:
# Get the parameters from the model
params = []
for layer in model.layers:
    param_list = layer.params()
    for name, param in param_list:
        params.append((layer, name, param))

# Initialize the optimizer (e.g., SGD, Adam, or Momentum)
optimizer = Adam(params, lr=0.001)  # Example using Adam optimizer

# Define the loss function
loss_fn = cross_entropy

In [23]:

# Train the model using the train function
train_losses, val_losses, train_accs, val_accs = train(
    model, train_loader, val_loader, loss_fn, optimizer, num_epochs=10
)

KeyboardInterrupt: 